In [1]:
from bokeh.plotting import figure
from bokeh.layouts import row, column, widgetbox, gridplot

from bokeh.io import curdoc
from bokeh.io import output_notebook, show, output_file

from bokeh.models import ColumnDataSource, HoverTool, TapTool, Range1d, OpenURL
from bokeh.models import LinearColorMapper , ColorBar
from bokeh.models.widgets import Select, Slider


import numpy as np

In [6]:
# =============================================
# THIS comes from INTERFACE
#
selected_exposure = '00000003'# args['exposure']
selected_arm = 'r' # args['arm']
selected_spectrograph =  4 # args['spectrograph']

# =============================================
# THIS comes from QLF.CFG
#
night = '20190101'

# ============================================
#  THIS READ yaml files
#
from scalar_metrics_v02 import LoadMetrics

cam = selected_arm+str(selected_spectrograph)
exp = selected_exposure # intentionaly redundant
lm = LoadMetrics(cam, exp, night);
metrics, tests  = lm.metrics, lm.tests 

# =============================================
# THIS is only to simplify the code understanding
#
countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

countpix loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countpix-r4-00000003.yaml
getbias loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getbias-r4-00000003.yaml
getrms loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getrms-r4-00000003.yaml
xwsigma loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-xwsigma-r4-00000003.yaml
countbins loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countbins-r4-00000003.yaml
integ loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-integ-r4-00000003.yaml
skycont loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skycont-r4-00000003.yaml
skypeak loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skypeak-r4-00000003.yaml
skyresid loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skyresid-r4-00000003.yaml

In [28]:
hist_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Frequency: </span>
            <span style="font-size: 13px; color: #515151">@hist</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">sigma: </span>
            <span style="font-size: 13px; color: #515151;">[@left, @right]</span>
        </div>
    </div>
"""
name_hi = 'NBINSHIGH'
name_med = 'NBINSMED'
name_low = 'NBINSLOW'

def bins_doane(data):
    # Dane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in countbins[sname]])
    b = b/sum([ (x - mean)**2 for x in countbins[sname]])**(1.5)
    try:
        return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
    except:
        return int(round(np.log2(ndata) + 1 ))
bins_hi = bins_doane(countbins[name_med])
bins_med = bins_doane(countbins[name_med])
bins_low = bins_doane(countbins[name_low])# formely: 17
hover = HoverTool(tooltips=hist_tooltip)

# ===
hist_hi, edges_hi = np.histogram(countbins[name_hi], bins = bins_hi)
source_hi = ColumnDataSource(data={
    'hist':hist_hi,
    'bottom':[0] *len(hist_hi),
    'left':edges_hi[:-1],
    'right':edges_hi[1:]
})

# ===
hist_med, edges_med = np.histogram(countbins[name_med], bins = bins_med)
source_med = ColumnDataSource(data={
    'hist':hist_med,
    'bottom':[0] *len(hist_med),
    'left':edges_med[:-1],
    'right':edges_med[1:]
})

# ===
hist_low, edges_low = np.histogram(countbins[name_low], bins = bins_low)
source_low = ColumnDataSource(data={
    'hist':hist_low,
    'bottom':[0] *len(hist_low),
    'left':edges_low[:-1],
    'right':edges_low[1:]
})



p = figure(title='NBINS',tools=[hover,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS',
           #y_axis_type='log',
            background_fill_color="white")

p.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_hi, 
        fill_color="dodgerblue", line_color="black", alpha=0.8,
       hover_fill_color='blue', hover_line_color='black', hover_alpha=0.8)

p.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_med, 
        fill_color="lightgreen", line_color="black", alpha=0.8,
       hover_fill_color='green', hover_line_color='black', hover_alpha=0.8)

p.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_low, 
        fill_color="tomato", line_color="black", alpha=0.8,
       hover_fill_color='red', hover_line_color='black', hover_alpha=0.8)
p.legend.location = "top_left"

#layout = columns(p)
output_notebook()
show(p)

/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in log2


Loading BokehJS ...

In [11]:
 countbins.keys()#['NPIXHIGH']# (blue), countbins[‘NPIXMED’] (green) and countbins[‘NPIXLOW’] (orang

dict_keys(['NBINSLOW', 'RA', 'NBINSLOW_AMP', 'NBINSHIGH_AMP', 'DEC', 'NBINSMED', 'NGOODFIBERS', 'NGOOD_ERR', 'NBINSHIGH', 'NBINSMED_AMP', 'NBINSHI_TEMP'])

In [14]:
countbins['NBINSHIGH'][:3]

[202.0, 2176.0, 198.0]

In [25]:
sname='NBINSHIGH'
mean=np.mean(countbins['NBINSMED'])
b = sum([ (x - mean)**3 for x in countbins[sname]])
b = b/sum([ (x - mean)**2 for x in countbins[sname]])**(1.5)
print( b)

-0.0407035701785
